In [35]:
#Accessing the data
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener
import json
import datetime as dt
from datetime import datetime, timedelta
import time
import os
import sys
import codecs
import io
import re

In [36]:
#loading the twitter API
def load_api():
    consumer_key = '9Pls4VOFjj8fPRNc5XLWXQbPF'
    consumer_secret = '8z4wjK5aJ9j1KQl5qxiskG5pL5hyutcbc45psahzEpuIrDFKVx'
    access_token = '854038913727516672-9Dt0RaINnUz0lLju6bV9pE7PVhxcAxB'
    access_secret = 'LlnU8ri5SWakSDpWmntWBoxCCJwhmsGvpi911FLy3Jg8H'
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    return auth

In [37]:
class MyStreamListener(tweepy.StreamListener):
    def __init__(self, time_limit=60):
        self.start_time = time.time()
        self.limit = time_limit
        self.saveFile = open('data1.json', 'w')
        super(MyStreamListener, self).__init__()

    def on_data(self, data):
        if (time.time() - self.start_time) < self.limit:
            self.saveFile.write(data)
            #self.saveFile.write('\n')
            return True
        else:
            self.saveFile.close()
            return False

In [44]:
auth = load_api()
myStream = tweepy.Stream(auth, listener=MyStreamListener(time_limit=600))
myStream.filter(track=['#Nike'])

In [40]:
with open('data.json') as json_data:
    dat = json_data.read()
    print dat

{"created_at":"Sat Sep 30 11:10:06 +0000 2017","id":914084938487492609,"id_str":"914084938487492609","text":"RT @NRGgg: #OverwatchSF Epic #iPhoneX #Giveaway\n-Tag Your Friends\n-RT, Like &amp; Follow @OverwatchSF\nClick Here:\u2026 ","source":"\u003ca href=\"https:\/\/gleam.io\" rel=\"nofollow\"\u003eGleam Competition App\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":400875034,"id_str":"400875034","name":"sscrash","screen_name":"sscrash1","location":"Pottsville, PA","url":null,"description":null,"translator_type":"none","protected":false,"verified":false,"followers_count":617,"friends_count":3825,"listed_count":109,"favourites_count":2204,"statuses_count":34414,"created_at":"Sat Oct 29 18:13:03 +0000 2011","utc_offset":-14400,"time_zone":"Eastern Time (US & Canada)","geo_enabled":true,"lang":"en","contributors_enabled":false,"is_transla

In [45]:
ccode = "US"

with codecs.open('data1.json', 'r') as fin, open('data1_filtered_test.txt', 'w') as fout:
    for line in fin:
        tweet = json.loads(line)
        if 'country_code' in tweet.keys() and 'retweet_count' in tweet.keys() and 'favorite_count' in tweet.keys():
                if tweet['lang'] == "en" and tweet['country_code'] == ccode and tweet['retweet_count'] > 0 and tweet['favorite_count'] > 0:
                    print tweet['text']
                    fout.write(tweet['text'].encode('ascii', errors='backslashreplace'))
        else:
                if tweet['lang'] == "en":
                    print tweet['text']
                    fout.write(tweet['text'].encode('ascii', errors='backslashreplace'))

RT @loveshoes3501: #NIKE AIR MORE UPTEMPO “FLAX”
RELEASE DATE: OCTOBER 14TH, 2017
$185
Style Code: AA4060-200
https://t.co/KCmkNYOh19 https…
New boots!!! The man in the shop  assure me I wasn't too old to pull them off 🤔🤔🤔🤔#tocolourful #nike #football #toolsofthetrade
Check out NFL Nike Team Vapor Jet 2.0 Gloves #Nike https://t.co/Z9MrsnWiRw via @eBay
NFL Nike Team Vapor Jet 2.0 Gloves #Nike https://t.co/Z9MrsnWiRw via @eBay https://t.co/wsKNSdmi49
NFL Nike Team Vapor Jet 2.0 Gloves #Nike https://t.co/Z9MrsnWiRw via @eBay https://t.co/deaTajo8Wp
NFL Nike Team Vapor Jet 2.0 Gloves #Nike https://t.co/Z9MrsnWiRw via @eBay https://t.co/8MYve6cvlB
NFL Nike Team Vapor Jet 2.0 Gloves #Nike https://t.co/Z9MrsnWiRw via @eBay https://t.co/dO00T2exQ5
Nike Air Jordan 12 Retro CNY Style 881427-122 Size 13 $280.00 via Amazon https://t.co/exem29UK9B #deals #nike #jordans
Goddess of speed and victory ⚡🔥 #Nike👑 #EnglishFestival #NewProfilePic https://t.co/T9YRmLeNhX
Lowest Ask: $229 Nike Flyknit Racer 

In [47]:
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens
 
with open('data1_filtered_test.txt', 'r') as fin, open('data1_token.txt', 'w') as fout:
    for line in fin:
        tweet = line
        #print(preprocess(tweet))
        fout.write(tweet)